In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Built in modules
import json
import os
import pickle
# Python modules
import numpy as np
# Machine Learning Frameworks
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
os.chdir('/content/drive/MyDrive/Recipe-NER')
model = tf.keras.models.load_model('ner_model_30e.h5')

In [4]:
with open('word2idx.json', 'r') as fcc_file:
    word2idx = json.load(fcc_file)
with open('tag2idx.json', 'r') as fcc_file:
    tag2idx = json.load(fcc_file)
with open("tags", "rb") as fp:   
    tags = pickle.load(fp)

In [5]:
# Place what you want to test in text
text = "In a separate small bowl make the dressing for the slaw Stir together the mayonnaise honey Dijon vinegar salt and pepper until smooth Add the dressing to the bowl with the slaw mix and green onions then stir to combine To make the sliders add a heaping 12 cup of BBQ beans to each bun then top with 12 cup of the slaw Enjoy Dice the onion and mince the garlic Add the onion garlic and olive oil to a large skillet and cook over medium heat until the onion is soft and translucent about 5 minutes Rinse and drain the pinto and black beans in a colander Add the drained beans BBQ sauce and smoked paprika to the skillet with the onions and garlic Stir to combine and then cook over medium-low heat until warmed through stir occasionally While the beans and BBQ sauce are heating prepare the creamy coleslaw Add the cabbage coleslaw mix to a large bowl Slice the green onions and add them to the bowl with the cabbage"
test = word_tokenize(text)
print(test)

['In', 'a', 'separate', 'small', 'bowl', 'make', 'the', 'dressing', 'for', 'the', 'slaw', 'Stir', 'together', 'the', 'mayonnaise', 'honey', 'Dijon', 'vinegar', 'salt', 'and', 'pepper', 'until', 'smooth', 'Add', 'the', 'dressing', 'to', 'the', 'bowl', 'with', 'the', 'slaw', 'mix', 'and', 'green', 'onions', 'then', 'stir', 'to', 'combine', 'To', 'make', 'the', 'sliders', 'add', 'a', 'heaping', '12', 'cup', 'of', 'BBQ', 'beans', 'to', 'each', 'bun', 'then', 'top', 'with', '12', 'cup', 'of', 'the', 'slaw', 'Enjoy', 'Dice', 'the', 'onion', 'and', 'mince', 'the', 'garlic', 'Add', 'the', 'onion', 'garlic', 'and', 'olive', 'oil', 'to', 'a', 'large', 'skillet', 'and', 'cook', 'over', 'medium', 'heat', 'until', 'the', 'onion', 'is', 'soft', 'and', 'translucent', 'about', '5', 'minutes', 'Rinse', 'and', 'drain', 'the', 'pinto', 'and', 'black', 'beans', 'in', 'a', 'colander', 'Add', 'the', 'drained', 'beans', 'BBQ', 'sauce', 'and', 'smoked', 'paprika', 'to', 'the', 'skillet', 'with', 'the', 'onion

In [6]:

# Same value as the one used in training
max_len = 900
preprocessed_text = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test]], 
                            padding="post", value=0, maxlen=max_len)

In [7]:
p = model.predict(np.array([preprocessed_text[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

1/1 [==============================] - 6s 6s/step
Word           ||Prediction
In             : O    
a              : O    
separate       : O    
small          : O    
bowl           : B-CO 
make           : O    
the            : O    
dressing       : O    
for            : O    
the            : O    
slaw           : O    
Stir           : O    
together       : O    
the            : O    
mayonnaise     : B-IN 
honey          : B-IN 
Dijon          : B-IN 
vinegar        : I-IN 
salt           : B-IN 
and            : O    
pepper         : B-IN 
until          : O    
smooth         : O    
Add            : O    
the            : O    
dressing       : O    
to             : O    
the            : O    
bowl           : O    
with           : O    
the            : O    
slaw           : O    
mix            : O    
and            : O    
green          : B-IN 
onions         : I-IN 
then           : O    
stir           : O    
to             : O    
combine        : O    
To